In [1]:
import pandas as pd
import requests
import urllib.request
import datetime
import time
import os
from ckanapi import RemoteCKAN

scidm = RemoteCKAN('https://scidm.nchc.org.tw', apikey='API_KEY')

In [2]:
name_list = []
url_list = []
resource_id_list = []

for data in scidm.call_action('package_show', {'id': 'scidm-usage-statistic'})['resources']:
    if '2019' in data['description'].split('_')[0]:
        name_list.append(data['description'])
        url_list.append(data['url'])
        resource_id_list.append(data['id'])
        print(data['id'], data['description'], data['url'])

b6f36828-128a-4dbe-bd64-8d195edd0e92 20190107_前10大被瀏覽ip.csv https://scidm.nchc.org.tw/dataset/3da79aba-6247-4ebe-ad31-2776f4c3e8ba/resource/b6f36828-128a-4dbe-bd64-8d195edd0e92/download/top10_ip_count.csv
824bc1a5-972f-408c-bb6b-79a084c37a15 20190107_前10大有下載動作ip.csv https://scidm.nchc.org.tw/dataset/3da79aba-6247-4ebe-ad31-2776f4c3e8ba/resource/824bc1a5-972f-408c-bb6b-79a084c37a15/download/top10_download_ip_count.csv
860598b6-f529-4585-9be4-346b9d020b46 20190107_前10大被瀏覽資料集.csv https://scidm.nchc.org.tw/dataset/3da79aba-6247-4ebe-ad31-2776f4c3e8ba/resource/860598b6-f529-4585-9be4-346b9d020b46/download/top10_dataset.csv
a66d55e5-1dfa-4b29-8668-18bf3636f9e1 20190107_前10大被下載資料集.csv https://scidm.nchc.org.tw/dataset/3da79aba-6247-4ebe-ad31-2776f4c3e8ba/resource/a66d55e5-1dfa-4b29-8668-18bf3636f9e1/download/top10_download_dataset.csv
d34bb32f-bcc2-4e51-9312-4d68f062981e 20190107_前10大瀏覽分類.csv https://scidm.nchc.org.tw/dataset/3da79aba-6247-4ebe-ad31-2776f4c3e8ba/resource/d34bb32f-bcc2-4e51-93

In [3]:
data_2019 = pd.DataFrame({'id':resource_id_list, 'name':name_list, 'url':url_list})
data_2019.head()

,id,name,url
0,b6f36828-128a-4dbe-bd64-8d195edd0e92,20190107_前10大被瀏覽ip.csv,https://scidm.nchc.org.tw/dataset/3da79aba-624...
1,824bc1a5-972f-408c-bb6b-79a084c37a15,20190107_前10大有下載動作ip.csv,https://scidm.nchc.org.tw/dataset/3da79aba-624...
2,860598b6-f529-4585-9be4-346b9d020b46,20190107_前10大被瀏覽資料集.csv,https://scidm.nchc.org.tw/dataset/3da79aba-624...
3,a66d55e5-1dfa-4b29-8668-18bf3636f9e1,20190107_前10大被下載資料集.csv,https://scidm.nchc.org.tw/dataset/3da79aba-624...
4,d34bb32f-bcc2-4e51-9312-4d68f062981e,20190107_前10大瀏覽分類.csv,https://scidm.nchc.org.tw/dataset/3da79aba-624...


In [4]:
len(data_2019)

400

In [10]:
def login(username, password):
    '''
    Login to CKAN.
    Returns a ``requests.Session`` instance with the CKAN
    session cookie.
    '''
    s = requests.Session()
    data = {'login': username, 'password': password}
    url = 'https://scidm.nchc.org.tw' + '/login_generic'
    r = s.post(url, data=data)
    if 'field-login' in r.text:
        # Response still contains login form
        raise RuntimeError('Login failed.')
    return s

def download_resource_data(session, data_2019):
    for name, url in zip(data_2019['name'], data_2019['url']):
        if '/' in name:
            name = name.replace('/','_')
        data = session.get(url)
        with open('/home/fish/桌面/log/2019/{}'.format(name), 'wb') as f:
            f.write(data.content)

if __name__ == '__main__':
    session = login('hsiaoyu', 'myps800510')
    download_resource_data(session, data_2019)

In [52]:
for id_ in list(data_2018['id']):
#     print(id_)
#     print(scidm.call_action('recourse_show', {'id': id_}))
    try:
        scidm.call_action('resource_delete', {'id': id_})
        time.sleep(10)
    except:
        pass